In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_train = pd.read_csv(r'/home/jovyan/mid-term/mid_term/fraud_credit/train.csv')
data_test = pd.read_csv(r'/home/jovyan/mid-term/mid_term/fraud_credit/test.csv')
data_sample = pd.read_csv(r'/home/jovyan/mid-term/mid_term/fraud_credit/sampleSubmission.csv')

print('Size of Training Set: ', data_train.shape[0], 'and', data_train.shape[1], 'features')
print('Size of Testing Set: ', data_test.shape[0] , 'and', data_test.shape[1], 'features')
print('Target Variable: class')

Size of Training Set:  108800 and 11 features
Size of Testing Set:  27200 and 10 features
Target Variable: class


In [3]:
alldata = pd.concat((data_train.loc[:,'user_id':'purchase_time'],
                     data_test.loc[:,'user_id':'purchase_time']))
alldata = alldata.reset_index(drop=True)
print('all_data:',alldata.shape)
data_test_id = data_test['user_id']
data_train['class'].value_counts(dropna=False)

all_data: (136000, 10)


False    98668
True     10132
Name: class, dtype: int64

In [4]:
misscol = []
for col in alldata.columns:
    if alldata[col].isnull().sum()>0:
        misscol.append(col)
        print(col,":" ,alldata[col].isnull().sum())

country : 19766


In [5]:
def fill_nan(data):
    data['country'] = data['country'].fillna('NA')
    return data

In [6]:
alldata = fill_nan(alldata)
print('處理完後NaN數量:',alldata.isnull().any().sum())

處理完後NaN數量: 0


In [7]:
print(alldata.columns)
alldata.drop(["user_id","device_id"],inplace = True,axis=1)
print(alldata.columns)

Index(['user_id', 'device_id', 'age', 'sex', 'browser', 'source', 'country',
       'purchase_value', 'signup_time', 'purchase_time'],
      dtype='object')
Index(['age', 'sex', 'browser', 'source', 'country', 'purchase_value',
       'signup_time', 'purchase_time'],
      dtype='object')


In [8]:
categorical = [col for col in alldata.columns if alldata[col].dtype == 'O']
for i in categorical:
    print(i,':',alldata[i].unique())

browser : ['Safari' 'Chrome' 'FireFox' 'IE' 'Opera']
source : ['Ads' 'Direct' 'SEO']
country : ['United States' 'European Union' 'NA' 'Germany' 'Japan' 'Hong Kong'
 'Netherlands' 'France' 'Norway' 'China' 'Poland' 'Spain' 'Italy'
 'United Kingdom' 'Korea Republic of' 'Brazil' 'Russian Federation'
 'Pakistan' 'Kenya' 'Denmark' 'Peru' 'Australia' 'Switzerland' 'Belgium'
 'Bulgaria' 'Viet Nam' 'Singapore' 'Czech Republic' 'Finland' 'India'
 'Sweden' 'Antigua and Barbuda' 'Turkey' 'Mexico' 'Latvia' 'Colombia'
 'Canada' 'Argentina' 'South Africa' 'Lebanon' 'Moldova Republic of'
 'Israel' 'Iran (ISLAMIC Republic Of)' 'Taiwan; Republic of China (ROC)'
 'Hungary' 'Syrian Arab Republic' 'Costa Rica' 'Greece' 'Iraq' 'Ecuador'
 'Croatia (LOCAL Name: Hrvatska)' 'Georgia' 'Thailand' 'Ukraine' 'Austria'
 'New Zealand' 'Chile' 'Libyan Arab Jamahiriya' 'Ireland' 'Indonesia'
 'Tunisia' 'Saudi Arabia' 'Portugal' 'Romania' 'El Salvador' 'Philippines'
 'Cambodia' 'Venezuela' 'Malaysia' 'Belarus' 'Slovenia

In [9]:
data_pro = alldata.copy()
data_pro  = alldata.select_dtypes(include=['object'])
data_1 = data_pro.copy()
data_1 = pd.get_dummies(data_1, columns=['browser','source','country'])
alldata = alldata.drop(['browser','source','country','signup_time','purchase_time'],axis=1)
finaldata = pd.concat([alldata,data_1], axis=1)
finaldata.head(5)

,age,sex,purchase_value,signup_time,purchase_time,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,33,0,48,2018-05-27 12:02:29,2018-08-19 14:09:55,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,26,1,50,2018-06-06 08:09:12,2018-09-09 01:26:43,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28,0,31,2018-03-29 18:24:17,2018-06-13 04:56:21,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,50,0,31,2018-06-18 02:49:33,2018-06-29 23:31:45,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,27,1,16,2018-01-31 18:26:38,2018-02-13 16:36:51,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
import pandas as pd
import numpy as np
import time
finaldata['purchase_time'] = pd.to_datetime(finaldata['purchase_time'])
finaldata['signup_time'] = pd.to_datetime(finaldata['signup_time'])

In [11]:
import pandas as pd
import numpy as np
import time

finaldata['purchase_season'] = finaldata['purchase_time'].dt.quarter.astype(str)
finaldata['purchase_month'] = finaldata['purchase_time'].dt.month.astype(str)
finaldata['purchase_week']=finaldata['purchase_time'].dt.week.astype(str)
finaldata['purchase_day'] = finaldata['purchase_time'].dt.day.astype(str)
finaldata['purchase_hour'] =finaldata['purchase_time'].dt.hour.astype(str)
finaldata['signup_season'] = finaldata['signup_time'].dt.quarter.astype(str)
finaldata['signup_month'] = finaldata['signup_time'].dt.month.astype(str)
finaldata['signup_week']=finaldata['signup_time'].dt.week.astype(str)
finaldata['signup_day'] = finaldata['signup_time'].dt.day.astype(str)
finaldata['signup_hour'] =finaldata['signup_time'].dt.hour.astype(str)
finaldata.head(5)

,age,sex,purchase_value,signup_time,purchase_time,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,...,purchase_season,purchase_month,purchase_week,purchase_day,purchase_hour,signup_season,signup_month,signup_week,signup_day,signup_hour
0,33,0,48,2018-05-27 12:02:29,2018-08-19 14:09:55,0,0,0,0,1,...,3,8,33,19,14,2,5,21,27,12
1,26,1,50,2018-06-06 08:09:12,2018-09-09 01:26:43,1,0,0,0,0,...,3,9,36,9,1,2,6,23,6,8
2,28,0,31,2018-03-29 18:24:17,2018-06-13 04:56:21,1,0,0,0,0,...,2,6,24,13,4,1,3,13,29,18
3,50,0,31,2018-06-18 02:49:33,2018-06-29 23:31:45,0,1,0,0,0,...,2,6,26,29,23,2,6,25,18,2
4,27,1,16,2018-01-31 18:26:38,2018-02-13 16:36:51,0,0,1,0,0,...,1,2,7,13,16,1,1,5,31,18


In [12]:
finaldata = finaldata.drop(['signup_time','purchase_time'],axis=1)

In [13]:
finaldata = pd.get_dummies(finaldata)

In [14]:
finaldata.describe()

,age,sex,purchase_value,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,source_Ads,source_Direct,...,signup_hour_21,signup_hour_22,signup_hour_23,signup_hour_3,signup_hour_4,signup_hour_5,signup_hour_6,signup_hour_7,signup_hour_8,signup_hour_9
count,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,...,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000,136000.000000
mean,33.146434,0.584169,36.960081,0.405912,0.162941,0.243316,0.024301,0.163529,0.396382,0.203007,...,0.041250,0.040713,0.040868,0.042449,0.040926,0.041574,0.041103,0.040882,0.042551,0.041250
std,8.621544,0.492866,18.335037,0.491069,0.369313,0.429086,0.153984,0.369849,0.489147,0.402239,...,0.198869,0.197626,0.197984,0.201611,0.198121,0.199613,0.198529,0.198018,0.201844,0.198869
min,18.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,1.000000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,39.000000,1.000000,49.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,76.000000,1.000000,154.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
X_train = finaldata[:108800]
X_test = finaldata[108800:]
y_train = data_train['class'].astype(str).map({'False': 0 ,'True':1})
print('train_y',y_train.shape)
print('test_data',X_test.shape)
X_train.head(5)

train_y (108800,)
test_data (27200, 412)


,age,sex,purchase_value,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,source_Ads,source_Direct,...,signup_hour_21,signup_hour_22,signup_hour_23,signup_hour_3,signup_hour_4,signup_hour_5,signup_hour_6,signup_hour_7,signup_hour_8,signup_hour_9
0,33,0,48,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,26,1,50,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,28,0,31,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,50,0,31,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,27,1,16,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

%matplotlib inline
rcParams['figure.figsize'] = (10, 5)

## 要開始預測囉

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=91058, test_size=0.3)


In [19]:
# 型別轉換，確保資料型態正確
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int)

In [20]:
def rmse(model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    RMSE_train = np.sqrt(mean_squared_error(y_train, pred_train))
    RMSE_val = np.sqrt(mean_squared_error(y_val, pred_val))
    return RMSE_train, RMSE_val

In [21]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%pylab inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [22]:
#DecisionTree

#clf = RandomForestClassifier()
# Train the model using the training sets
#clf.fit(X_train, y_train)

In [23]:
# Predict Output
#from sklearn import metrics
# make prediction here
# y_pred = '''make prediction on testing dataset here'''
#y_pred_t_DTRe = clf.predict(X_train)
#y_pred_v_DTRe = clf.predict(X_val)
# visualize the result

#%matplotlib inline

#from sklearn import metrics
#print(metrics.classification_report(y_train, y_pred_t_DTRe))
#print(metrics.classification_report(y_val, y_pred_v_DTRe))

In [35]:
eval_set = [(X_val, y_val)]
clf = XGBClassifier( learning_rate =0.01,
                     n_estimators=20,
                     max_depth=5,
                     min_child_weight=8,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8)
clf.fit(X_train,
          y_train,
          early_stopping_rounds=10,
          eval_metric="auc",
          eval_set=eval_set,
          verbose=True)

[0]	validation_0-auc:0.76734
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.774087
[2]	validation_0-auc:0.774101
[3]	validation_0-auc:0.77398
[4]	validation_0-auc:0.77398
[5]	validation_0-auc:0.77398
[6]	validation_0-auc:0.773957
[7]	validation_0-auc:0.774094
[8]	validation_0-auc:0.773842
[9]	validation_0-auc:0.773841
[10]	validation_0-auc:0.773841
[11]	validation_0-auc:0.773917
[12]	validation_0-auc:0.773837
Stopping. Best iteration:
[2]	validation_0-auc:0.774101



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1,
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=8, missing=None, n_estimators=20, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [36]:
test_y_pred = clf.predict(X_test)
test_y_pred=test_y_pred.astype(np.int)

In [37]:
submit_file = pd.DataFrame({'user_id': data_test_id, 'class': test_y_pred})
submit_file.to_csv('./fraud_091058_2.csv', index=False)